In [ ]:
import subprocess

import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

# get peak binding sites

In [ ]:
ddir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac'
ts = [6,8,10,12,14,17,21]
for ti in ts:
    peak_sites_data = []
    for AvsC in ['A', 'C']:
        print(ti, AvsC)

        wkdir = ddir + f'/fimo_P{ti}_{AvsC}'
        
        f_genome_order = '../mm10.sorted.chrom.sizes'
        fin   = wkdir + '/fimo.tsv'
        fin2  = ddir + f'/P{ti}_{AvsC}_peaks.bed'
        
        ftmp  = wkdir + '/fimo_subset_tmp.tsv'
        ftmp2 = wkdir + '/fimo_subset_tmp.bed'
        ftmp3 = wkdir + '/fimo_subset_tmp_sorted.bed'
        fout  = wkdir + '/fimo_subset_closest_peaks.bed'
        
        # subset the files
        command = ['grep', 'FOXP', 'fimo.tsv'] 
        with open(ftmp, 'w') as f:
            subprocess.run(command, cwd=wkdir, stdout=f, stderr=subprocess.PIPE, text=True)
            
        # read and sort (to bed)
        df = pd.read_csv(ftmp, sep='\t', header=None)
        dfout = df.groupby([2,3,4]).first().reset_index()[[2,3,4,1]]
        dfout.to_csv(ftmp2, sep='\t', index=False, header=False)
        
        # bedtools sort -g ../mm10.sorted.chrom.sizes -i fimo_subset_regionsort.bed > fimo_subset_regionsort2.bed
        command = ['bedtools', 'sort', '-g', f_genome_order, '-i', ftmp2]
        with open(ftmp3, 'w') as f:
            subprocess.run(command, cwd=wkdir, stdout=f, stderr=subprocess.PIPE, text=True)

        # bedtools closest -a fimo_subset_regionsort2.bed -b ../P8_A_peaks_ngene.bed > fimo_subset_regionsort_ngene.bed
        command = ['bedtools', 'closest', '-a', ftmp3, '-b', fin2]
        with open(fout, 'w') as f:
            subprocess.run(command, cwd=wkdir, stdout=f, stderr=subprocess.PIPE, text=True)
            
        df = pd.read_csv(fout, sep='\t', header=None)
        df['peak'] = np.array([f'{chrom}:{start}-{end}' for chrom, start, end in zip(df[4], df[5], df[6])])
        peak_sites = df.groupby('peak').size().sort_values(ascending=False)
        peak_sites_data.append(peak_sites)
    break

In [ ]:
%%time
# RNA data 
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21.h5ad'
adata = sc.read(f_rna)
adata

In [ ]:
%%time
adatas_pk = []
# for ti in [6,8,10,12,14,17,21]:
for ti in [6]:
    print(ti)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P{ti}_ATAC_L23_peakbycell_filtered.h5ad'
    adata_pk = sc.read(f)
    adata_pk.obs = adata_pk.obs.set_index('cell')
    print(adata_pk.var['peak'][:3].values)
    
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
# get cells based on RNA
# get matrix based on ATAC - CPM or CP10k

qs_avc = []
l2fc_avc = []
# for cond_order, obssub in adata.obs.groupby('cond_order'):
for cond_order in [0]:
    obssub = adata[adata.obs['cond_order']==cond_order].obs
    
    # get sub
    adatasub = adata[obssub.index]
    adata_pk = adatas_pk[cond_order]
    
    # get A vs C 
    x = adatasub.obsm['pca_p17on'][...,0]
    cond_a = x < np.percentile(x,20)
    cond_c = x > np.percentile(x,80)
    cells_a = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_a].obs.index.values])
    cells_c = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_c].obs.index.values])
    
    cells_rna = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub.obs.index.values])
    adatasub.obs.index = cells_rna
    
    cells_atac = adata_pk.obs.index.values
    cells_mult = np.intersect1d(cells_rna, cells_atac) 
    
    adata_pk_a = adata_pk[np.intersect1d(cells_a, cells_atac)]
    adata_pk_c = adata_pk[np.intersect1d(cells_c, cells_atac)]
    
    mat_a = np.log10(1+adata_pk_a.X[...]) # layers['lognorm'][...]
    mat_c = np.log10(1+adata_pk_c.X[...]) # adatasub_c.layers['lognorm'][...]
    
    # DEGs
    lfc = np.log2(10)*(np.mean(mat_c, axis=0) - np.mean(mat_a, axis=0)) # log2FC (log10CPM as raw counts)
    ts, ps = stats.ttest_ind(mat_a, mat_c)
    rs, qs, _, _ = multipletests(np.nan_to_num(ps, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    
    num_sig = np.sum(np.logical_and(qs < 0.05, np.abs(lfc) > 1))
    print(cond_order, mat_a.shape, mat_c.shape, num_sig)
    
    # save this
    qs_avc.append(qs)
    l2fc_avc.append(lfc)
    
    # break
    
qs_avc = np.array(qs_avc)
l2fc_avc = np.array(l2fc_avc)

In [ ]:
l2fc_th = 1
alpha_th = 0.05
cond_sig   = np.logical_and(qs_avc < alpha_th, np.abs(l2fc_avc) > l2fc_th)
cond_sig_c = np.logical_and(qs_avc < alpha_th, l2fc_avc >  l2fc_th)
cond_sig_a = np.logical_and(qs_avc < alpha_th, l2fc_avc < -l2fc_th)

print('num AvsC-DEGs per cond:\t', cond_sig.sum(axis=1))
print('num A-DEGs per cond:\t',    cond_sig_a.sum(axis=1))
print('num C-DEGs per cond:\t',    cond_sig_c.sum(axis=1))

In [ ]:
x = adatasub[cells_mult].obsm['pca_p17on'][...,0]

adata_pk_a = adata_pk[:,cond_sig_a][cells_mult]
adata_pk_c = adata_pk[:,cond_sig_c][cells_mult]

mat_pk_a = np.log10(1+adata_pk_a.X[...])
mat_pk_c = np.log10(1+adata_pk_c.X[...])
y_a = np.mean(mat_pk_a, axis=1)
y_c = np.mean(mat_pk_c, axis=1)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5)# , facecolor='none')
ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5)# , facecolor='none')
r_a, p  = stats.spearmanr(x, y_a)
r_c, p  = stats.spearmanr(x, y_c)

ax.set_title(f'r_a={r_a:.4f}; r_c={r_c:.4f}')
ax.set_xlabel('RNA PC1 (A-C axis)')
ax.set_ylabel('ATAC')
ax.legend(bbox_to_anchor=(1,1))

sns.despine(ax=ax)
ax.grid(False)

plt.show()

# combine the two

In [ ]:
tmp2a = adata_pk_a.var.set_index('peak')
tmp2a = tmp2a.join(peak_sites_data[0].to_frame('num_sites')).fillna(0) 

tmp2c = adata_pk_c.var.set_index('peak')
tmp2c = tmp2c.join(peak_sites_data[1].to_frame('num_sites')).fillna(0) 

In [ ]:
fig, axs = plt.subplots(1,8,figsize=(8*4,4), sharex=True, sharey=True)
for i in range(8):
    ax = axs[i]
    cond_a = (tmp2a['num_sites'].values == i)
    cond_c = (tmp2c['num_sites'].values == i)
    y_a = np.mean(mat_pk_a[:, cond_a], axis=1)
    y_c = np.mean(mat_pk_c[:, cond_c], axis=1)

    ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5)# , facecolor='none')
    ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5)# , facecolor='none')

    ax.set_title(f'num. FOXP sites = {i}')
    ax.set_xlabel('RNA PC1 (A-C axis)')
    ax.set_ylabel('ATAC')
    ax.legend(bbox_to_anchor=(1,1))

    sns.despine(ax=ax)
    ax.grid(False)

plt.show()